In [ ]:
!rm "./unet.pkl"
# remove ?dl=0, man
!wget -O unet.pkl https://www.dropbox.com/s/mj2m9gteu5ou6sk/unet.pkl?dl=0

# Preparing model

In [33]:
import torch
import torch.nn.functional as F

import numpy as np

from PIL import Image

from skimage.io import imshow
from skimage.io import imsave

import matplotlib.pyplot as plt

In [34]:
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=False)

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [36]:
def prep_image(image, crop_size, size_y, size_x):
    # Resize the image
    prepd_image = image.resize((size_y, size_x))
    # Crop the image to remove the border black pixels
    borders = (crop_size, crop_size, size_x - crop_size, size_y - crop_size)
    prepd_image = prepd_image.crop(borders)
    return prepd_image

In [37]:
def make_mask_from_path(image_path, model):
    img = Image.open(image_path)
    img = prep_image(img,20,168,168)
    img = np.asarray(img) / 255

    model.to(device)
    tensor = torch.unsqueeze(torch.FloatTensor(np.rollaxis(img, 2, 0)), 0).to(device)
    with torch.no_grad():
        mask = model(tensor)
    # mask = F.interpolate(mask.cpu(), size=1024, mode='bilinear', align_corners=False)
    mask = np.rollaxis(torch.round(mask[0].cpu()).numpy(), 0, 3)
    img = img - img * mask + mask * 0.1
    return img, mask

In [38]:
import pickle

with open('unet.pkl', 'rb') as f:
  model = pickle.load(f)

In [39]:
!pip install pytelegrambotapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
import telebot

from telebot import types

from pathlib import Path

# Telegram Bot

In [41]:
bot = telebot.TeleBot()

In [42]:
@bot.message_handler(commands = ['start'])
def smth(message):
    markup = types.InlineKeyboardMarkup()
    bot.send_message(message.from_user.id, "Send the pic to get the mask!", reply_markup = markup)

In [43]:
@bot.message_handler(content_types='photo')
def pic_reply(message):
    Path(f'files/{message.chat.id}/').mkdir(parents=True, exist_ok=True)
    file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    
    src = f'files/{message.chat.id}/' + file_info.file_path.replace('photos/', '')
    with open(src, 'wb') as new_file:
        new_file.write(downloaded_file)
    
    image, mask = make_mask_from_path(src, model)
    
    src = f'files/{message.chat.id}/' + 'image.jpg'
    imsave(src, image)
    with open(src, 'rb') as f:
      bot.send_photo(message.from_user.id, f)

    src = f'files/{message.chat.id}/' + 'mask.jpg'
    imsave(src, mask)
    with open(src, 'rb') as f:
      bot.send_photo(message.from_user.id, f)
    

In [44]:
bot.polling(none_stop=True, interval=0)

# PS